In [99]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
datapath = '/home/bigdatas16/1101.csv'
data = pd.read_csv(datapath)
def MA(data,day):
    dataList = range(data['Date'].size)
    tmpList = []
    for i in dataList:
        if i - day + 1 >= 0:
            tmp = data['Close'][i-day+1:i+1].mean()
            tmpList.append(tmp)
        elif i - day + 1 < 0:
            tmp = 0
            tmpList.append(tmp)
    tmpSeries = pd.Series(tmpList)
    data['MA'+str(day)]=tmpSeries

# 乖離率(BIAS_lag1)
def BIAS_lag(data,day):
    dataList = range(data['Date'].size)
    tmpList = []
    tmpList.append(0)
    for item in dataList:
        if item-day+1 >= 0:
            tmp = (data['Close'][item]-data['MA'+str(day)][item])/(data['MA'+str(day)][item])*100
            tmpList.append(tmp)
        elif item - day + 1 < 0:
            tmp = 0
            tmpList.append(tmp)         
    tmpSeries = pd.Series(tmpList)
    data['BIAS'+str(day)] = tmpSeries
for i in range(24):
    MA(data,1+i)
    BIAS_lag(data,1+i)
def UP_DOWN(data):
    data_day_number = range(data['Date'].size)
    tmpList = []
    
    for item in data_day_number:
        spread = data['Close'][item] - data['Open'][item]
        if spread > 0 :
            tmp = 1
        elif spread <= 0:
            tmp = 0
 
        tmpList.append(tmp)
    
    tmpSeries = pd.Series(tmpList)
    data['UP_DOWN']=tmpSeries
UP_DOWN(data)
data1 = data.drop(['Date', 'Open', 'High','Low','Close','Volume_n','Volume_m','return','PE','MA1','MA2','MA3','MA4','MA5','MA6','MA7','MA8','MA9','MA10','MA11','MA12','MA13','MA14','MA15','MA16','MA17','MA18','MA19','MA20','MA21','MA22','MA23','MA24'], axis=1)
data1 = data1.fillna(-1)
cols = data1.columns.tolist()
cols[24]
data1 = data1[[cols[24]] + cols[0:23]]
X = data1.iloc[:,1:]
Y = data1.iloc[:,0:1] 
#hyperparameter tuning
test_error = []
numTrees_maxDepth = []
for j in range(1,7,1):
    for i in range(1,21,1):
        n = len(data['Date'])/2
        sql_sc = SQLContext(sc)
        train_data = data1.iloc[:n,:]
        train_sql = sql_sc.createDataFrame(train_data)
        test_data = data1.iloc[n:,:]
        test_sql = sql_sc.createDataFrame(test_data)
        assembler1 = VectorAssembler(inputCols=["BIAS1", "BIAS2", "BIAS3","BIAS4","BIAS5","BIAS6"
                                               ,"BIAS7","BIAS8","BIAS9","BIAS10","BIAS11","BIAS12"
                                               ,"BIAS13","BIAS14","BIAS15","BIAS16","BIAS17","BIAS18"
                                               ,"BIAS19","BIAS20","BIAS21","BIAS22","BIAS23"],outputCol="features")
        train_sql = assembler1.transform(train_sql)
        labelIndexer = StringIndexer(inputCol = "UP_DOWN", outputCol="indexedLabel").fit(train_sql)
        featureIndexer = VectorIndexer(inputCol = "features", outputCol="indexedFeatures").fit(train_sql)
        rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",numTrees = i,maxDepth = j)
        pipeline = Pipeline(stages=[labelIndexer,featureIndexer,rf])
        model = pipeline.fit(train_sql)
        assembler = VectorAssembler(inputCols=["BIAS1", "BIAS2", "BIAS3","BIAS4","BIAS5","BIAS6"
                                               ,"BIAS7","BIAS8","BIAS9","BIAS10","BIAS11","BIAS12"
                                               ,"BIAS13","BIAS14","BIAS15","BIAS16","BIAS17","BIAS18"
                                               ,"BIAS19","BIAS20","BIAS21","BIAS22","BIAS23"],outputCol="features")
        test_sql = assembler.transform(test_sql)
        predictions = model.transform(test_sql)
        evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")#, metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        test_error.append(1.0 - accuracy)
        numTrees_maxDepth.append([i,j])
        #print("Test Error = %g , numTrees = %g , maxDepth = %g" % (1.0 - accuracy , i, j))
#for i in range(len(test_error)):
    #print(numTrees_maxDepth[i],test_error[i],i)
mintest = np.min(test_error)
c = []
for i in range(len(test_error)):
    c.append(np.min(test_error) == test_error[i])
    
for i in range(len(test_error)):    
    if c[i] == True:
        print(numTrees_maxDepth[i],mintest)

Populating the interactive namespace from numpy and matplotlib
([3, 6], 0.52620053773870556)
